# Tutorial 2: Basic NLP Pipeline - Spacy
The goal of the first tutorial is to create an NLP pipeline and get familiar with the common NLP modules. You will be able to handle most of the tasks with pandas as well as the nltk module, which are the basis for many NLP projects. Please use a (Jupyter) notebook for your code, for keeping things clean, performative and easy to annotate. Believe me, it's worth the extra effort!



## How to find Information
Since you will not directly be able to solve all tasks just from what we provide in the lectures, we will provide you with some relevant materials and online-tutorials. Don't worry if you cannot solve a task; ask in the GitHub discussion forum or wait until we talk about it in the tutorial session.

### Python 
If you need a little refreshment of your Python skills or want to quickly find well-structured information on certain aspects of Python, check out the tutorials and documentation on [W3 Schools](https://www.w3schools.com/python/default.asp). 

### Pandas 
Pandas is THE tool for tabular data in python. It comes with a lot of useful features, but is not always self-explanatory. However, there is an accelent [documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/index.html) and a superb [tutorial](https://www.w3schools.com/python/pandas/default.asp) on Pandas, and we reccomend to check both out, especially the first few lessens of the tutorial! 

### NLTK
NLTK, the natural language toolkit, as the name suggests, provides virtually everything you need for natural language processing in python. Although Spacy gets more popular quickly, NLTK is still the most used module for most preprocessing and analysis tasks. Find the documentation on the [official website](https://www.nltk.org/), but thanks to the popularity, almost every problem was already discussed and every question answered somewhere on the internet and almost always, a google search for a description of a problem or an error message leads to a solution. 


## Spacy

Spacy is the latest and "best" NLP Python Framework. However, it is even further away from "standard" python than NLTK. Luckily, there is a very good tutorial on [the official website](https://course.spacy.io/en), not far away from where you can find the [excellent documentation](https://spacy.io/api). We recommend the introduction chapters of the documentation and the first two chapters of the Tutorial.

## 1: Installation and Import
### 1.1 Installation and Import of NLP Modules

The first Task is the same as in the first tutorial:
Make sure that the required modules "pandas", "numpy", "nltk", "sklearn", "spacy" and "re" are imported and get the version numbers of the modules.

Hint: Use <code>! pip install NAMEOFMODULE</code> to run command line / shell commands from your python jupyter notebook and install modules without having to use your terminal. 

In [1]:
import pandas as pd
print ("pandas", pd.__version__)

import numpy as np
print ("numpy", np.__version__)

import nltk
print ("nltk", nltk.__version__)

import re
print ("re", re.__version__)

import sklearn
print ("sklearn", sklearn.__version__)

import spacy
print ("spacy", spacy.__version__)


pandas 1.3.4
numpy 1.22.3
nltk 3.7
re 2.2.1
sklearn 1.0.1
spacy 2.3.2


### 1.2 Import of "Quality-of-Life Modules"
Install "pandarallel". Import pandarallel's method "pandarallel" (for parallelization in pandas) and initialize it with pandarallel.initialize(). Consult pandarallel's documentation or google if you need help with using pandarallel.

In [2]:
# quality of life improvements
from pandarallel import pandarallel  # parallelization
pandarallel.initialize()

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


## 2: Import Data
Again, we will work with the jokes dataset from the first tutorial.

Read in the attached .json files as pandas dataframes and merge them into a single dataframe. When doing so, make sure that the source of the data is preserved as a key.

In [3]:
dfs = [pd.read_json("stupidstuff.json")[:100], pd.read_json("reddit.json")[:100], pd.read_json("wocka.json")[:100]]
data = pd.concat(dfs, keys=["stupidstuff", "reddit", "wocka"])
data.head()

body  \
stupidstuff 0  A blackjack dealer and a player with a thirtee...   
            1  At a dinner party, several of the guests were ...   
            2  One day this cop pulls over a blonde for speed...   
            3  Three women are about to be executed for crime...   
            4  A girl came skipping home FROM school one day....   

                   category id  rating  score title  
stupidstuff 0      Children  1    2.63    NaN   NaN  
            1  Blonde Jokes  2    2.57    NaN   NaN  
            2  Blonde Jokes  3    3.09    NaN   NaN  
            3  Blonde Jokes  4    4.10    NaN   NaN  
            4  Blonde Jokes  5    4.30    NaN   NaN

## 3: Data Preprocessing
This time, we want to perform our preprocessing using Spacy. 

### 3.1 Prepare Spacy
Load a model/pipeline (find a fitting model [here](https://spacy.io/models)), it should be a "medium" sized model (size of models is usually noted at the end of the model name), to be sure that it includes all the functionality you need.


In [4]:
import spacy
nlp = spacy.load("en_core_web_md")

In [5]:
# test:
print((nlp("This is a test. Wonderful!"))[5])

Wonderful


### 3.2 Clean up text
Prepare the data so that the "body" column contains the entire text of the joke and that no format tokens (e.g. "\n") are included. Use Spacy pattern matching, if possible!

In [6]:
data.loc["reddit"]

,body,category,id,rating,score,title
0,"Now I have to say ""Leroy can you please paint the fence?""",NaN,5tz52q,NaN,1.0,I hate how you cant even say black paint anymore
1,Pizza doesn't scream when you put it in the oven .\n\nI'm so sorry.,NaN,5tz4dd,NaN,0.0,What's the difference between a Jew in Nazi Germany and pizza ?
2,"...and being there really helped me learn about American culture. So I visited a shop and as I was leaving, the Shopkeeper said ""Have a nice day!"" But I didn't so I sued him.",NaN,5tz319,NaN,0.0,I recently went to America....
3,"A Sunday school teacher is concerned that his students might be a little confused about Jesus, so he asks his class, “Where is Jesus today?”\nBrian raises his hand and says, “He’s in Heaven.”\n\nSusan answers, “He’s in my heart.”\n\nLittle Johnny waves his hand furiously and blurts out, “He’s in our bathroom!”\n\nThe teacher is surprised by this answer and asks Little Johnny how he knows this.\n\n“Well,” Little Johnny says, “every morning, my Dad gets up, bangs on the bathroom door and yells ‘Jesus Christ, are you still in there?'”",NaN,5tz2wj,NaN,1.0,"Brian raises his hand and says, “He’s in Heaven.”"
4,He got caught trying to sell the two books to a freshman.,NaN,5tz1pc,NaN,0.0,"You hear about the University book store worker who was charged for stealing $20,000 worth of books?"
...,...,...,...,...,...,...
95,"So if you're a great driver, look out for women who are turning.",NaN,5txmyo,NaN,16.0,A lot of women turn into great drivers...
96,Blonde #1: Oh how I hate when he brings me flowers. All night I'll be on my back with my legs in the air.\nBlonde #2: Don't you own a vase?,NaN,5txmlh,NaN,4.0,Valentines Day Flowers
97,Espn2,NaN,5txm9l,NaN,2.0,#1 Uconn women's team going for historic 100th consecutive team in a sold out stadium against #6 South Carolina
98,"He breaks into a house to look for money and guns and finds a young couple in bed.\n\nHe orders the guy out of bed and ties him to a chair, while tying the girl to the bed he gets on top of her, kisses her neck, then gets up and goes into the bathroom. While he's in there, the husband tells his wife:\n\n""Listen, this guy's an escaped convict, look at his clothes! He probably spent lots of time in jail and hasn't seen a woman in years. I saw how he kissed your neck."" If he wants sex, don't resist, don't complain, do whatever he tells you. Satisfy him no matter how much he nauseates you. This guy is probably very dangerous. If he gets angry, he'll kill us. Be strong, honey. I love you.""\n\nTo which his wife responds: ""He wasn't kissing my neck. He was whispering in my ear. He told me he was gay, thought you were cute, and asked me if we had any vaseline. I told him it was in the bathroom. Be strong honey. I love you too!!""",NaN,5txluy,NaN,6.0,A man escapes from prison where he has been for 15 years


In [7]:
data.loc["reddit"]["body"] = data.loc["reddit"].parallel_apply(lambda row: row["title"]+" "+row["body"], axis=1)
data.loc["reddit"]

<ipython-input-7-20f353993307>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc["reddit"]["body"] = data.loc["reddit"].parallel_apply(lambda row: row["title"]+" "+row["body"], axis=1)


,body,category,id,rating,score,title
0,"I hate how you cant even say black paint anymore Now I have to say ""Leroy can you please paint the fence?""",NaN,5tz52q,NaN,1.0,I hate how you cant even say black paint anymore
1,What's the difference between a Jew in Nazi Germany and pizza ? Pizza doesn't scream when you put it in the oven .\n\nI'm so sorry.,NaN,5tz4dd,NaN,0.0,What's the difference between a Jew in Nazi Germany and pizza ?
2,"I recently went to America.... ...and being there really helped me learn about American culture. So I visited a shop and as I was leaving, the Shopkeeper said ""Have a nice day!"" But I didn't so I sued him.",NaN,5tz319,NaN,0.0,I recently went to America....
3,"Brian raises his hand and says, “He’s in Heaven.” A Sunday school teacher is concerned that his students might be a little confused about Jesus, so he asks his class, “Where is Jesus today?”\nBrian raises his hand and says, “He’s in Heaven.”\n\nSusan answers, “He’s in my heart.”\n\nLittle Johnny waves his hand furiously and blurts out, “He’s in our bathroom!”\n\nThe teacher is surprised by this answer and asks Little Johnny how he knows this.\n\n“Well,” Little Johnny says, “every morning, my Dad gets up, bangs on the bathroom door and yells ‘Jesus Christ, are you still in there?'”",NaN,5tz2wj,NaN,1.0,"Brian raises his hand and says, “He’s in Heaven.”"
4,"You hear about the University book store worker who was charged for stealing $20,000 worth of books? He got caught trying to sell the two books to a freshman.",NaN,5tz1pc,NaN,0.0,"You hear about the University book store worker who was charged for stealing $20,000 worth of books?"
...,...,...,...,...,...,...
95,"A lot of women turn into great drivers... So if you're a great driver, look out for women who are turning.",NaN,5txmyo,NaN,16.0,A lot of women turn into great drivers...
96,Valentines Day Flowers Blonde #1: Oh how I hate when he brings me flowers. All night I'll be on my back with my legs in the air.\nBlonde #2: Don't you own a vase?,NaN,5txmlh,NaN,4.0,Valentines Day Flowers
97,#1 Uconn women's team going for historic 100th consecutive team in a sold out stadium against #6 South Carolina Espn2,NaN,5txm9l,NaN,2.0,#1 Uconn women's team going for historic 100th consecutive team in a sold out stadium against #6 South Carolina
98,"A man escapes from prison where he has been for 15 years He breaks into a house to look for money and guns and finds a young couple in bed.\n\nHe orders the guy out of bed and ties him to a chair, while tying the girl to the bed he gets on top of her, kisses her neck, then gets up and goes into the bathroom. While he's in there, the husband tells his wife:\n\n""Listen, this guy's an escaped convict, look at his clothes! He probably spent lots of time in jail and hasn't seen a woman in years. I saw how he kissed your neck."" If he wants sex, don't resist, don't complain, do whatever he tells you. Satisfy him no matter how much he nauseates you. This guy is probably very dangerous. If he gets angry, he'll kill us. Be strong, honey. I love you.""\n\nTo which his wife responds: ""He wasn't kissing my neck. He was whispering in my ear. He told me he was gay, thought you were cute, and asked me if we had any vaseline. I told him it was in the bathroom. Be strong honey. I love you too!!""",NaN,5txluy,NaN,6.0,A man escapes from prison where he has been for 15 years


In [8]:
# now with spacy https://spacy.io/usage/rule-based-matching :

# Import the Matcher
from spacy.matcher import Matcher

# Initialize the matcher with the shared vocab
matcher = Matcher(nlp.vocab)

# Add the pattern(s) to the matcher
pattern1 = [{"ORTH": "\n"}] #does not work!
pattern2 = [{"IS_SPACE": True}]
matcher.add("non-word_pattern", [pattern2])

doc = nlp("this is a test. \n New lines are fun!\r")

matches = matcher(doc)


print(matches)


# better readable:
for match_id, start, end in matches:
    # Get the matched span
    matched_span = doc[start-2:end]
    print(matched_span.text)


[(14302392117694624264, 5, 6), (14302392117694624264, 11, 12)]
test. 
 
fun!


In [37]:
def del_nontext_tokens(inp_text):
    doc = nlp(inp_text)
    new_doc = doc.text
    names = []
    
    # add a matcher that matches pattern 2 from above
    matcher.add('non-word', None, [pattern2]) 
    matches = matcher(doc) 
    
    # get "text version" of all matches (similar to printing them)
    for match_id, start, end in matches: 
        span = doc[start:end] 
        names.append(span.text) 
    
    # replace all matches with spaces
    for name in names: 
        new_doc = new_doc.replace(name,'')
    
    return new_doc

In [10]:
del_nontext_tokens("this is a \n test.")

'this is a test.'

In [11]:
data["body"] = data.parallel_apply(lambda row: del_nontext_tokens(row["body"]), axis=1)
data

body  \
stupidstuff 0   A blackjack dealer and a player with a thirtee...   
            1   At a dinner party, several of the guests were ...   
            2   One day this cop pulls over a blonde for speed...   
            3   Three women are about to be executed for crime...   
            4   A girl came skipping home FROM school one day....   
...                                                           ...   
wocka       95  A first grade teacher collected well-known pro...   
            96  Alabama:At Least We're not MississippiAlaska:1...   
            97  Q: What is your date of birth? A: July fifteen...   
            98  (Setting: A furious lightsaber duel is underwa...   
            99  At one point during a game, the coach said to ...   

                    category   id  rating  score                 title  
stupidstuff 0       Children    1    2.63    NaN                   NaN  
            1   Blonde Jokes    2    2.57    NaN                   NaN  
            2   Blonde Jokes    3    3.09    NaN                   NaN  
            3   Blonde Jokes    4    4.10    NaN                   NaN  
            4   Blonde Jokes    5    4.30    NaN                   NaN  
...                      ...  ...     ...    ...                   ...  
wocka       95  Other / Misc  103     NaN    NaN              Proverbs  
            96  Other / Misc  104     NaN    NaN         State Slogans  
            97  Other / Misc  105     NaN    NaN  Things Said In Court  
            98  Other / Misc  106     NaN    NaN  Real Skywalker Scene  
            99        Sports  107     NaN    NaN            Good Sport  

[300 rows x 6 columns]

### 3.3 Tokenization
Use Spacy to tokenize the jokes. Save the results in a column "spacy_tokens".

In [12]:
# we want a list of tokens:
print(list(nlp("this is a test.")))
print(type(list(nlp("this is a test."))[2]))

[this, is, a, test, .]
<class 'spacy.tokens.token.Token'>


In [13]:
# but we can work around this problem:
[a.text for a in list(nlp("this is a test."))]

['this', 'is', 'a', 'test', '.']

In [14]:
data["tokens"] = data.apply(lambda row: [a.text for a in list(nlp(str(row["body"])))], axis=1)

In [15]:
data

body  \
stupidstuff 0   A blackjack dealer and a player with a thirtee...   
            1   At a dinner party, several of the guests were ...   
            2   One day this cop pulls over a blonde for speed...   
            3   Three women are about to be executed for crime...   
            4   A girl came skipping home FROM school one day....   
...                                                           ...   
wocka       95  A first grade teacher collected well-known pro...   
            96  Alabama:At Least We're not MississippiAlaska:1...   
            97  Q: What is your date of birth? A: July fifteen...   
            98  (Setting: A furious lightsaber duel is underwa...   
            99  At one point during a game, the coach said to ...   

                    category   id  rating  score                 title  \
stupidstuff 0       Children    1    2.63    NaN                   NaN   
            1   Blonde Jokes    2    2.57    NaN                   NaN   
            2   Blonde Jokes    3    3.09    NaN                   NaN   
            3   Blonde Jokes    4    4.10    NaN                   NaN   
            4   Blonde Jokes    5    4.30    NaN                   NaN   
...                      ...  ...     ...    ...                   ...   
wocka       95  Other / Misc  103     NaN    NaN              Proverbs   
            96  Other / Misc  104     NaN    NaN         State Slogans   
            97  Other / Misc  105     NaN    NaN  Things Said In Court   
            98  Other / Misc  106     NaN    NaN  Real Skywalker Scene   
            99        Sports  107     NaN    NaN            Good Sport   

                                                           tokens  
stupidstuff 0   [A, blackjack, dealer, and, a, player, with, a...  
            1   [At, a, dinner, party, ,, several, of, the, gu...  
            2   [One, day, this, cop, pulls, over, a, blonde, ...  
            3   [Three, women, are, about, to, be, executed, f...  
            4   [A, girl, came, skipping, home, FROM, school, ...  
...                                                           ...  
wocka       95  [A, first, grade, teacher, collected, well, -,...  
            96  [Alabama, :, At, Least, We, 're, not, Mississi...  
            97  [Q, :, What, is, your, date, of, birth, ?, A, ...  
            98  [(, Setting, :, A, furious, lightsaber, duel, ...  
            99  [At, one, point, during, a, game, ,, the, coac...  

[300 rows x 7 columns]

### 3.4 Stopword removal
Remove all English stopwords from the generated tokens. Use Spacy.

In [16]:
all_stopwords = nlp.Defaults.stop_words

In [17]:
def remove_stopwords_from_list(list_in):
    return [a for a in list_in if a not in all_stopwords]

In [19]:
print(remove_stopwords_from_list(data.tokens[1]))

['At', 'dinner', 'party', ',', 'guests', 'arguing', 'men', 'women', 'trustworthy', '.', "'", 'No', 'woman', ',', "'", 'said', 'man', ',', 'scornfully', ',', "'", 'secret', '.', "'", "'", 'I', 'know', ',', "'", 'answered', 'blonde', 'woman', 'guest', '.', "'", 'I', 'kept', 'age', 'secret', 'I', '-', '.', "'", "'", 'You', 'let', 'day', ',', "'", 'man', 'insisted', '.', "'", 'I', 'hardly', 'think', '!', "'", 'responded', 'blonde', 'lady', '.', "'", 'When', 'woman', 'kept', 'secret', '-', 'seven', 'years', ',', 'forever', '.', "'"]


In [22]:
data["no_stop"] = data.parallel_apply(lambda row: remove_stopwords_from_list(row["tokens"]), axis=1)

In [23]:
data

body  \
stupidstuff 0   A blackjack dealer and a player with a thirtee...   
            1   At a dinner party, several of the guests were ...   
            2   One day this cop pulls over a blonde for speed...   
            3   Three women are about to be executed for crime...   
            4   A girl came skipping home FROM school one day....   
...                                                           ...   
wocka       95  A first grade teacher collected well-known pro...   
            96  Alabama:At Least We're not MississippiAlaska:1...   
            97  Q: What is your date of birth? A: July fifteen...   
            98  (Setting: A furious lightsaber duel is underwa...   
            99  At one point during a game, the coach said to ...   

                    category   id  rating  score                 title  \
stupidstuff 0       Children    1    2.63    NaN                   NaN   
            1   Blonde Jokes    2    2.57    NaN                   NaN   
            2   Blonde Jokes    3    3.09    NaN                   NaN   
            3   Blonde Jokes    4    4.10    NaN                   NaN   
            4   Blonde Jokes    5    4.30    NaN                   NaN   
...                      ...  ...     ...    ...                   ...   
wocka       95  Other / Misc  103     NaN    NaN              Proverbs   
            96  Other / Misc  104     NaN    NaN         State Slogans   
            97  Other / Misc  105     NaN    NaN  Things Said In Court   
            98  Other / Misc  106     NaN    NaN  Real Skywalker Scene   
            99        Sports  107     NaN    NaN            Good Sport   

                                                           tokens  \
stupidstuff 0   [A, blackjack, dealer, and, a, player, with, a...   
            1   [At, a, dinner, party, ,, several, of, the, gu...   
            2   [One, day, this, cop, pulls, over, a, blonde, ...   
            3   [Three, women, are, about, to, be, executed, f...   
            4   [A, girl, came, skipping, home, FROM, school, ...   
...                                                           ...   
wocka       95  [A, first, grade, teacher, collected, well, -,...   
            96  [Alabama, :, At, Least, We, 're, not, Mississi...   
            97  [Q, :, What, is, your, date, of, birth, ?, A, ...   
            98  [(, Setting, :, A, furious, lightsaber, duel, ...   
            99  [At, one, point, during, a, game, ,, the, coac...   

                                                          no_stop  
stupidstuff 0   [A, blackjack, dealer, player, thirteen, count...  
            1   [At, dinner, party, ,, guests, arguing, men, w...  
            2   [One, day, cop, pulls, blonde, speeding, ., Th...  
            3   [Three, women, executed, crimes, ., One, brune...  
            4   [A, girl, came, skipping, home, FROM, school, ...  
...                                                           ...  
wocka       95  [A, grade, teacher, collected, -, known, prove...  
            96  [Alabama, :, At, Least, We, MississippiAlaska:...  
            97  [Q, :, What, date, birth, ?, A, :, July, fifte...  
            98  [(, Setting, :, A, furious, lightsaber, duel, ...  
            99  [At, point, game, ,, coach, said, young, playe...  

[300 rows x 8 columns]

### 3.5 POS Tagging
Determine POS tags for the tokenized texts and store them in a "pos" column. Again, use Spacy.

In [24]:
doc= nlp("this is a test.")
for t in doc:
    print(t.pos_)

DET
AUX
DET
NOUN
PUNCT


In [25]:
data["pos"] = data.parallel_apply(lambda row: [t.pos_ for t in nlp(row["body"])], axis=1)

In [31]:
data

body  \
stupidstuff 0   A blackjack dealer and a player with a thirtee...   
            1   At a dinner party, several of the guests were ...   
            2   One day this cop pulls over a blonde for speed...   
            3   Three women are about to be executed for crime...   
            4   A girl came skipping home FROM school one day....   
...                                                           ...   
wocka       95  A first grade teacher collected well-known pro...   
            96  Alabama:At Least We're not MississippiAlaska:1...   
            97  Q: What is your date of birth? A: July fifteen...   
            98  (Setting: A furious lightsaber duel is underwa...   
            99  At one point during a game, the coach said to ...   

                    category   id  rating  score                 title  \
stupidstuff 0       Children    1    2.63    NaN                   NaN   
            1   Blonde Jokes    2    2.57    NaN                   NaN   
            2   Blonde Jokes    3    3.09    NaN                   NaN   
            3   Blonde Jokes    4    4.10    NaN                   NaN   
            4   Blonde Jokes    5    4.30    NaN                   NaN   
...                      ...  ...     ...    ...                   ...   
wocka       95  Other / Misc  103     NaN    NaN              Proverbs   
            96  Other / Misc  104     NaN    NaN         State Slogans   
            97  Other / Misc  105     NaN    NaN  Things Said In Court   
            98  Other / Misc  106     NaN    NaN  Real Skywalker Scene   
            99        Sports  107     NaN    NaN            Good Sport   

                                                           tokens  \
stupidstuff 0   [A, blackjack, dealer, and, a, player, with, a...   
            1   [At, a, dinner, party, ,, several, of, the, gu...   
            2   [One, day, this, cop, pulls, over, a, blonde, ...   
            3   [Three, women, are, about, to, be, executed, f...   
            4   [A, girl, came, skipping, home, FROM, school, ...   
...                                                           ...   
wocka       95  [A, first, grade, teacher, collected, well, -,...   
            96  [Alabama, :, At, Least, We, 're, not, Mississi...   
            97  [Q, :, What, is, your, date, of, birth, ?, A, ...   
            98  [(, Setting, :, A, furious, lightsaber, duel, ...   
            99  [At, one, point, during, a, game, ,, the, coac...   

                                                          no_stop  \
stupidstuff 0   [A, blackjack, dealer, player, thirteen, count...   
            1   [At, dinner, party, ,, guests, arguing, men, w...   
            2   [One, day, cop, pulls, blonde, speeding, ., Th...   
            3   [Three, women, executed, crimes, ., One, brune...   
            4   [A, girl, came, skipping, home, FROM, school, ...   
...                                                           ...   
wocka       95  [A, grade, teacher, collected, -, known, prove...   
            96  [Alabama, :, At, Least, We, MississippiAlaska:...   
            97  [Q, :, What, date, birth, ?, A, :, July, fifte...   
            98  [(, Setting, :, A, furious, lightsaber, duel, ...   
            99  [At, point, game, ,, coach, said, young, playe...   

                                                              pos  \
stupidstuff 0   [DET, NOUN, NOUN, CCONJ, DET, NOUN, ADP, DET, ...   
            1   [ADP, DET, NOUN, NOUN, PUNCT, ADJ, ADP, DET, N...   
            2   [NUM, NOUN, DET, NOUN, VERB, ADP, DET, NOUN, A...   
            3   [NUM, NOUN, AUX, ADJ, PART, AUX, VERB, ADP, NO...   
            4   [DET, NOUN, VERB, VERB, ADV, ADP, NOUN, NUM, N...   
...                                                           ...   
wocka       95  [DET, ADJ, NOUN, NOUN, VERB, ADV, PUNCT, VERB,...   
            96  [PROPN, PUNCT, ADP, ADJ, PRON, AUX, PART, PROP...   
            97  [X, PUNCT, PRON, AUX, DET, NOUN, ADP, NOUN, PU..

### 3.6 Lemmatization
Lemmatize the tokens of the texts and store the specific lemmas in a column "lemmata". Use the Spacy Lemmatizer.

In [27]:
# simple version
doc = nlp("this is a test.")
for t in doc:
    print(t.lemma_)

this
be
a
test
.


In [28]:
data["lemma"] = data.parallel_apply(lambda row: [t.lemma_ for t in nlp(row["body"])], axis=1)

In [36]:
data

body  \
stupidstuff 0   A blackjack dealer and a player with a thirtee...   
            1   At a dinner party, several of the guests were ...   
            2   One day this cop pulls over a blonde for speed...   
            3   Three women are about to be executed for crime...   
            4   A girl came skipping home FROM school one day....   
...                                                           ...   
wocka       95  A first grade teacher collected well-known pro...   
            96  Alabama:At Least We're not MississippiAlaska:1...   
            97  Q: What is your date of birth? A: July fifteen...   
            98  (Setting: A furious lightsaber duel is underwa...   
            99  At one point during a game, the coach said to ...   

                    category   id  rating  score                 title  \
stupidstuff 0       Children    1    2.63    NaN                   NaN   
            1   Blonde Jokes    2    2.57    NaN                   NaN   
            2   Blonde Jokes    3    3.09    NaN                   NaN   
            3   Blonde Jokes    4    4.10    NaN                   NaN   
            4   Blonde Jokes    5    4.30    NaN                   NaN   
...                      ...  ...     ...    ...                   ...   
wocka       95  Other / Misc  103     NaN    NaN              Proverbs   
            96  Other / Misc  104     NaN    NaN         State Slogans   
            97  Other / Misc  105     NaN    NaN  Things Said In Court   
            98  Other / Misc  106     NaN    NaN  Real Skywalker Scene   
            99        Sports  107     NaN    NaN            Good Sport   

                                                           tokens  \
stupidstuff 0   [A, blackjack, dealer, and, a, player, with, a...   
            1   [At, a, dinner, party, ,, several, of, the, gu...   
            2   [One, day, this, cop, pulls, over, a, blonde, ...   
            3   [Three, women, are, about, to, be, executed, f...   
            4   [A, girl, came, skipping, home, FROM, school, ...   
...                                                           ...   
wocka       95  [A, first, grade, teacher, collected, well, -,...   
            96  [Alabama, :, At, Least, We, 're, not, Mississi...   
            97  [Q, :, What, is, your, date, of, birth, ?, A, ...   
            98  [(, Setting, :, A, furious, lightsaber, duel, ...   
            99  [At, one, point, during, a, game, ,, the, coac...   

                                                          no_stop  \
stupidstuff 0   [A, blackjack, dealer, player, thirteen, count...   
            1   [At, dinner, party, ,, guests, arguing, men, w...   
            2   [One, day, cop, pulls, blonde, speeding, ., Th...   
            3   [Three, women, executed, crimes, ., One, brune...   
            4   [A, girl, came, skipping, home, FROM, school, ...   
...                                                           ...   
wocka       95  [A, grade, teacher, collected, -, known, prove...   
            96  [Alabama, :, At, Least, We, MississippiAlaska:...   
            97  [Q, :, What, date, birth, ?, A, :, July, fifte...   
            98  [(, Setting, :, A, furious, lightsaber, duel, ...   
            99  [At, point, game, ,, coach, said, young, playe...   

                                                              pos  \
stupidstuff 0   [DET, NOUN, NOUN, CCONJ, DET, NOUN, ADP, DET, ...   
            1   [ADP, DET, NOUN, NOUN, PUNCT, ADJ, ADP, DET, N...   
            2   [NUM, NOUN, DET, NOUN, VERB, ADP, DET, NOUN, A...   
            3   [NUM, NOUN, AUX, ADJ, PART, AUX, VERB, ADP, NO...   
            4   [DET, NOUN, VERB, VERB, ADV, ADP, NOUN, NUM, N...   
...                                                           ...   
wocka       95  [DET, ADJ, NOUN, NOUN, VERB, ADV, PUNCT, VERB,...   
            96  [PROPN, PUNCT, ADP, ADJ, PRON, AUX, PART, PROP...   
            97  [X, PUNCT, PRON, AUX, DET, NOUN, ADP, NOUN, PU..

## 4: How was that?
What do you think? How was working with Spacy compared to working with NLTK? Prepare some thoughts to share in out tutorial!

## 5 Bonus

Play around with some more Spacy functionalties! Feel free to show us what you did in the tutorial!